In [1]:
from mbdata import MetabaseDataApi as m

In [7]:
session_params = dict(user='a.b.c@company.com',
                      password='*******',
                      url='https://your.metabase.com/'
                      )

mbapi = m(**session_params)



sql = """
select 'step 1' as source, 'step 2' as target, '100' as value
union all
select 'step 2' as source, 'step 3' as target, '60' as value
union all
select 'step 2' as source, 'step 3 alt' as target, '10' as value
union all
select 'step 2' as source, 'exit' as target, '10' as value
union all
select 'step 3' as source, 'exit' as target, '60' as value
union all
select 'step 3 alt' as source, 'exit' as target, '10' as value
union all
select 'step 1' as source, 'exit' as target, '20' as value
"""

# get data by card id
card_params = dict(card_id='39', export_format='json')
data = mbapi.get_query_data(sql, database_id=4)
print(data)


[{'source': 'step 1', 'target': 'step 2', 'value': '100'}, {'source': 'step 2', 'target': 'step 3', 'value': '60'}, {'source': 'step 2', 'target': 'step 3 alt', 'value': '10'}, {'source': 'step 2', 'target': 'exit', 'value': '10'}, {'source': 'step 3', 'target': 'exit', 'value': '60'}, {'source': 'step 3 alt', 'target': 'exit', 'value': '10'}, {'source': 'step 1', 'target': 'exit', 'value': '20'}]


In [9]:
import plotly.graph_objects as go
import random


def show_sankey(data, title):
  """data is a list of rows from sql of form
    (source_label, target_label, value )
  """

  labels = list(set([row['source'] for row in data] + [row['target'] for row in data] ))
  source = [labels.index(row['source']) for row in data] 
  target = [labels.index(row['target']) for row in data] 
  value =  [row['value'] for row in data] 


  color_map = {}
  for label in labels: 
    if 'exit' in label:
      color_map[label] = 'red' 

  color_randomizer = 2345#random.randint(1,10000)
  print(color_randomizer)
  color_index = []
  def color_map(color_index,label):
    if 'exit' in label:
       return 'red' 
    #add if not exist
    color_index = set(color_index)
    #print(label[1:])
    color_index.add(label[1:])
    color_index = list(color_index)
    #print(color_index)

    label_without_position = label[1:]
    random.seed(color_index.index(label_without_position)+color_randomizer)
    color = f'rgb({random.randint(0, 255)},{random.randint(0, 255)},{random.randint(0, 255)},0.9)'
    return color
  
  target_strings = [row['target'] for row in data] 
  node_colors = [color_map(labels,label) for label in labels]
  flow_colors = [color_map(labels,t).replace('0.9', '0.4') for t in target_strings]


  fig = go.Figure(data=[go.Sankey(
      node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = labels,
        color = node_colors
      ),
      link = dict(
        source = source,
        target = target,
        value = value,
        color = flow_colors
    ))])

  fig.update_layout(title_text=title, font_size=10)
  fig.show()

show_sankey(data, 'my sankey example')

2345
